In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. Load all datasets
coverage_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\coverage_data.csv", encoding="latin1")
incidence_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\incidence_rate_data.csv", encoding="latin1")
reported_cases_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\reported_cases_data.csv", encoding="latin1")
vaccine_intro_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\vaccine_introduction_data.csv", encoding="latin1")
vaccine_schedule_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\vaccine_schedule_data.csv", encoding="latin1")

datasets = {
    "Coverage": coverage_df,
    "Incidence": incidence_df,
    "Reported Cases": reported_cases_df,
    "Vaccine Introduction": vaccine_intro_df,
    "Vaccine Schedule": vaccine_schedule_df
}

# 2. Initial Missing/Duplicate Check
print("\nInitial Missing Values Check")
for name, df in datasets.items():
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print(f"\n{name} missing values:\n{missing}")
    else:
        print(f"\n{name}: No missing values")

print("\nInitial Duplicate Check")
for name, df in datasets.items():
    print(f"{name}: {df.duplicated().sum()} duplicate rows")

# 3. Display Columns and Datatypes
print("\nColumns and Data Types")
for name, df in datasets.items():
    print(f"\n{name} columns and datatypes:")
    print(df.dtypes)



Initial Missing Values Check

Coverage missing values:
CODE                                  1
NAME                               1275
YEAR                                  1
ANTIGEN                               1
ANTIGEN_DESCRIPTION                   1
COVERAGE_CATEGORY                     1
COVERAGE_CATEGORY_DESCRIPTION         1
TARGET_NUMBER                    320829
DOSES                            320532
COVERAGE                         169382
dtype: int64

Incidence missing values:
CODE                       1
NAME                       1
YEAR                       1
DISEASE                    1
DISEASE_DESCRIPTION        1
DENOMINATOR                1
INCIDENCE_RATE         23362
dtype: int64

Reported Cases missing values:
CODE                       1
NAME                       1
YEAR                       1
DISEASE                    1
DISEASE_DESCRIPTION        1
CASES                  19400
dtype: int64

Vaccine Introduction missing values:
COUNTRYNAME    1
WHO_REGION    

In [12]:
import pandas as pd
import numpy as np

# Load all datasets
coverage_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\coverage_data.csv", encoding="latin1")
incidence_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\incidence_rate_data.csv", encoding="latin1")
reported_cases_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\reported_cases_data.csv", encoding="latin1")
vaccine_intro_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\vaccine_introduction_data.csv", encoding="latin1")
vaccine_schedule_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\Raw_data\vaccine_schedule_data.csv", encoding="latin1")

# Dictionary for convenience
datasets = {
    "Coverage": coverage_df,
    "Incidence": incidence_df,
    "Reported Cases": reported_cases_df,
    "Vaccine Introduction": vaccine_intro_df,
    "Vaccine Schedule": vaccine_schedule_df
}

# Enhanced preprocessing function
def preprocess_dataset_enhanced(df, numeric_cols=[], categorical_cols=[], fill_unknown_cols=[], year_col=None):
    # Convert empty strings and other common null representations to NaN
    df.replace('', np.nan, inplace=True)
    df.replace('NA', np.nan, inplace=True)
    df.replace('NULL', np.nan, inplace=True)
    
    # Drop rows if Year is missing
    if year_col and year_col in df.columns:
        df.dropna(subset=[year_col], inplace=True)
        # Ensure year column is of integer type
        df[year_col] = df[year_col].astype(int)
    
    # Fill numeric columns with median
    for col in numeric_cols:
        if col in df.columns:
            # First, coerce column to numeric, which will turn invalid values into NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col].fillna(df[col].median(), inplace=True)
    
    # Fill categorical columns with "Unknown"
    for col in categorical_cols:
        if col in df.columns:
            df[col].fillna('Unknown', inplace=True)
    
    # Fill optional object columns like TARGETPOP with "Unknown"
    for col in fill_unknown_cols:
        if col in df.columns:
            df[col].fillna('Unknown', inplace=True)
    
    # Strip leading/trailing spaces from all object columns
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()
    
    return df

# Apply the enhanced preprocessing to each dataset
coverage_df = preprocess_dataset_enhanced(
    coverage_df,
    numeric_cols=['TARGET_NUMBER','DOSES','COVERAGE'],
    categorical_cols=['NAME','ANTIGEN_DESCRIPTION','COVERAGE_CATEGORY_DESCRIPTION'],
    year_col='YEAR'
)

incidence_df = preprocess_dataset_enhanced(
    incidence_df,
    numeric_cols=['INCIDENCE_RATE'],
    categorical_cols=['NAME','DISEASE_DESCRIPTION'],
    year_col='YEAR'
)

reported_cases_df = preprocess_dataset_enhanced(
    reported_cases_df,
    numeric_cols=['CASES'],
    categorical_cols=['NAME','DISEASE_DESCRIPTION'],
    year_col='YEAR'
)

vaccine_intro_df = preprocess_dataset_enhanced(
    vaccine_intro_df,
    categorical_cols=['COUNTRYNAME','WHO_REGION','DESCRIPTION','INTRO'],
    year_col='YEAR'
)

vaccine_schedule_df = preprocess_dataset_enhanced(
    vaccine_schedule_df,
    numeric_cols=['SCHEDULEROUNDS'],
    categorical_cols=['VACCINE_DESCRIPTION','GEOAREA','SOURCECOMMENT'],
    fill_unknown_cols=['TARGETPOP','AGEADMINISTERED','TARGETPOP_DESCRIPTION'],
    year_col='YEAR'
)

# Final check: Missing values after enhanced preprocessing
for name, df in datasets.items():
    print(f"\n{name} dataset missing values after enhanced preprocessing:")
    print(df.isnull().sum())


Coverage dataset missing values after enhanced preprocessing:
GROUP                            0
CODE                             0
NAME                             0
YEAR                             0
ANTIGEN                          0
ANTIGEN_DESCRIPTION              0
COVERAGE_CATEGORY                0
COVERAGE_CATEGORY_DESCRIPTION    0
TARGET_NUMBER                    0
DOSES                            0
COVERAGE                         0
dtype: int64

Incidence dataset missing values after enhanced preprocessing:
GROUP                  0
CODE                   0
NAME                   0
YEAR                   0
DISEASE                0
DISEASE_DESCRIPTION    0
DENOMINATOR            0
INCIDENCE_RATE         0
dtype: int64

Reported Cases dataset missing values after enhanced preprocessing:
GROUP                  0
CODE                   0
NAME                   0
YEAR                   0
DISEASE                0
DISEASE_DESCRIPTION    0
CASES                  0
dtype: int64

Vacc

In [13]:
# Dictionary of datasets
datasets = {
    "Coverage": coverage_df,
    "Incidence": incidence_df,
    "Reported Cases": reported_cases_df,
    "Vaccine Introduction": vaccine_intro_df,
    "Vaccine Schedule": vaccine_schedule_df
}

# Base path to save files
base_path = r"C:\Labmentix\Vaccination project\Datasets"

# Loop through datasets and save
for name, df in datasets.items():
    file_path = f"{base_path}\\{name.replace(' ', '_').lower()}.csv"
    df.to_csv(file_path, index=False)

print("All datasets have been saved successfully!")


All datasets have been saved successfully!


In [19]:
# Drop the database
cursor.execute("DROP DATABASE IF EXISTS vaccination")
print("Database dropped successfully!")

NameError: name 'cursor' is not defined

In [20]:
import mysql.connector
import pandas as pd

# MySQL connection (connect without database first to create it)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Srisql@8"  # replace with your MySQL password
)
cursor = conn.cursor()

# Create database
cursor.execute("CREATE DATABASE IF NOT EXISTS Vaccination")
cursor.execute("USE Vaccination")

# Define table creation queries
table_queries = {
    "coverage": """
        CREATE TABLE IF NOT EXISTS coverage (
            `GROUP` VARCHAR(50),
            CODE VARCHAR(30),
            NAME VARCHAR(100),
            YEAR INT,
            ANTIGEN VARCHAR(50),
            ANTIGEN_DESCRIPTION VARCHAR(100),
            COVERAGE_CATEGORY VARCHAR(50),
            COVERAGE_CATEGORY_DESCRIPTION VARCHAR(100),
            TARGET_NUMBER FLOAT,
            DOSES FLOAT,
            COVERAGE FLOAT
        )
    """,
    "incidence": """
        CREATE TABLE IF NOT EXISTS incidence (
            `GROUP` VARCHAR(50),
            CODE VARCHAR(30),
            NAME VARCHAR(100),
            YEAR INT,
            DISEASE VARCHAR(50),
            DISEASE_DESCRIPTION VARCHAR(100),
            DENOMINATOR VARCHAR(50),
            INCIDENCE_RATE FLOAT
        )
    """,
    "reported_cases": """
        CREATE TABLE IF NOT EXISTS reported_cases (
            `GROUP` VARCHAR(50),
            CODE VARCHAR(30),
            NAME VARCHAR(100),
            YEAR INT,
            DISEASE VARCHAR(50),
            DISEASE_DESCRIPTION VARCHAR(100),
            CASES FLOAT
        )
    """,
    "vaccine_introduction": """
        CREATE TABLE IF NOT EXISTS vaccine_introduction (
            ISO_3_CODE VARCHAR(10),
            COUNTRYNAME VARCHAR(100),
            WHO_REGION VARCHAR(50),
            YEAR INT,
            DESCRIPTION VARCHAR(100),
            INTRO VARCHAR(50)
        )
    """,
    "vaccine_schedule": """
        CREATE TABLE IF NOT EXISTS vaccine_schedule (
            ISO_3_CODE VARCHAR(10),
            COUNTRYNAME VARCHAR(100),
            WHO_REGION VARCHAR(50),
            YEAR INT,
            VACCINECODE VARCHAR(20),
            VACCINE_DESCRIPTION VARCHAR(100),
            SCHEDULEROUNDS FLOAT,
            TARGETPOP VARCHAR(50),
            TARGETPOP_DESCRIPTION VARCHAR(50),
            GEOAREA VARCHAR(50),
            AGEADMINISTERED VARCHAR(50),
            SOURCECOMMENT TEXT
        )
    """
}

# Execute table creation
for query in table_queries.values():
    cursor.execute(query)

# Load cleaned CSVs
coverage_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\coverage.csv")
incidence_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\incidence.csv")
reported_cases_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\reported_cases.csv")
vaccine_intro_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\vaccine_introduction.csv")
vaccine_schedule_df = pd.read_csv(r"C:\Labmentix\Vaccination project\Datasets\vaccine_schedule.csv")

# Insert data function
def insert_data(df, table_name):
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns = ", ".join([f"`{col}`" for col in df.columns])
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    for i, row in df.iterrows():
        cursor.execute(sql, tuple(row))
    conn.commit()
    print(f"{table_name} data inserted successfully!")

# Insert all datasets
insert_data(coverage_df, "coverage")
insert_data(incidence_df, "incidence")
insert_data(reported_cases_df, "reported_cases")
insert_data(vaccine_intro_df, "vaccine_introduction")
insert_data(vaccine_schedule_df, "vaccine_schedule")

# Close connection
cursor.close()
conn.close()
print("All datasets inserted and connection closed successfully!")


coverage data inserted successfully!
incidence data inserted successfully!
reported_cases data inserted successfully!
vaccine_introduction data inserted successfully!
vaccine_schedule data inserted successfully!
All datasets inserted and connection closed successfully!


In [25]:
import mysql.connector

# Connect to the Vaccination database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Srisql@8",  # replace with your MySQL password
    database="Vaccination"
)
cursor = conn.cursor()

# Columns to create indexes on
desired_indexes = {
    "coverage": ["YEAR", "CODE"],
    "incidence": ["YEAR", "CODE", "DISEASE"],
    "reported_cases": ["YEAR", "CODE", "DISEASE"],
    "vaccine_schedule": ["ISO_3_CODE", "YEAR"],
    "vaccine_introduction": ["ISO_3_CODE", "YEAR"]
}

print("Creating optimized indexes for faster Tableau visualization...")

for table, cols in desired_indexes.items():
    try:
        # Get existing columns
        cursor.execute(f"DESCRIBE {table}")
        existing_cols = [row[0] for row in cursor.fetchall()]

        for col in cols:
            if col in existing_cols:
                idx_name = f"idx_{table}_{col.lower()}"
                # Check if index already exists
                cursor.execute(f"SHOW INDEX FROM {table} WHERE Key_name = '{idx_name}'")
                if cursor.fetchone():
                    print(f"Index {idx_name} already exists on {table}, skipping.")
                else:
                    cursor.execute(f"CREATE INDEX {idx_name} ON {table}({col})")
                    print(f"Created index {idx_name} on {table}({col})")
            else:
                print(f"Column {col} not found in {table}, skipping.")
    except mysql.connector.Error as e:
        print(f"Error processing {table}: {e}")

conn.commit()
cursor.close()
conn.close()
print("\nIndex optimization complete! Your tables are now ready for fast Tableau queries.")


Creating optimized indexes for faster Tableau visualization...
Created index idx_coverage_year on coverage(YEAR)
Created index idx_coverage_code on coverage(CODE)
Created index idx_incidence_year on incidence(YEAR)
Created index idx_incidence_code on incidence(CODE)
Created index idx_incidence_disease on incidence(DISEASE)
Created index idx_reported_cases_year on reported_cases(YEAR)
Created index idx_reported_cases_code on reported_cases(CODE)
Created index idx_reported_cases_disease on reported_cases(DISEASE)
Created index idx_vaccine_schedule_iso_3_code on vaccine_schedule(ISO_3_CODE)
Created index idx_vaccine_schedule_year on vaccine_schedule(YEAR)
Created index idx_vaccine_introduction_iso_3_code on vaccine_introduction(ISO_3_CODE)
Created index idx_vaccine_introduction_year on vaccine_introduction(YEAR)

Index optimization complete! Your tables are now ready for fast Tableau queries.
